In [103]:
import os
cache_dir = "/scratches/dialfs/alta/hln35/.cache"
os.environ['TRANSFORMERS_CACHE'] = '/scratches/dialfs/alta/hln35/.cache'

In [104]:
model_small = "google/flan-t5-small"

In [105]:
from datasets import load_dataset

data_points = load_dataset("ai2_arc", "ARC-Easy", cache_dir=cache_dir)

In [106]:
data_points

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 2251
    })
    test: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 2376
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 570
    })
})

In [107]:
data_points["train"][0]

{'id': 'Mercury_7220990',
 'question': 'Which factor will most likely cause a person to develop a fever?',
 'choices': {'text': ['a leg muscle relaxing after exercise',
   'a bacterial population in the bloodstream',
   'several viral particles on the skin',
   'carbohydrates being digested in the stomach'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'B'}

In [108]:
prefix = "context: for each questions there are a few choices of answer. answer this question by choosing the best choice: "


def preprocess_function(data_points):
    # inputs = [prefix + example[source_lang] for example in examples["translation"]]
    # targets = [example[target_lang] for example in examples["translation"]]
    # model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    # model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    inputs = []
    for i in range(len(data_points["question"])):
        q = data_points["question"][i]
        choices = ""
        choice = ""
        for t in range(len(data_points["choices"][i]["label"])):
            choices += data_points["choices"][i]["label"][t] + " " + data_points["choices"][i]["text"][t] + ". "
            
        text = prefix + q + "Choices: " + choices
        inputs.append(text)
    model_inputs = tokenizer_large(inputs, truncation=True)
    
    return model_inputs

In [134]:
import evaluate

metric = evaluate.load("accuracy", cache_dir=cache_dir)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [110]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForQuestionAnswering, AutoModel, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_small)
model = AutoModelForSeq2SeqLM.from_pretrained(model_small)
model1 = AutoModel.from_pretrained(model_small)
model2 = AutoModelForQuestionAnswering.from_pretrained(model_small)



Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [111]:
model


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [112]:
model1

T5Model(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): Linear(in_features=

In [113]:
model2

T5ForQuestionAnswering(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): Line

In [114]:
import json, re

In [229]:
question = data_points['train'][0]["question"]
choices = ""
choice = ""
ans = ""
for t in range(len(data_points['train'][0]["choices"]["label"])):
    choices += data_points['train'][0]["choices"]["label"][t] + " " + data_points['train'][0]["choices"]["text"][t] + ". "
    
text = prefix + question + "Choices: " + choices
inputs = tokenizer(text, truncation=True, return_tensors="pt").input_ids
preds_tokenized = model(inputs.to("cuda"), decoder_input_ids=torch.tensor([[model.config.decoder_start_token_id,]]).to("cuda")) 

In [117]:
tokenizer(['This answer is A'])

{'input_ids': [[100, 1525, 19, 71, 1]], 'attention_mask': [[1, 1, 1, 1, 1]]}

In [118]:
tokenizer(['This answer is B'])

{'input_ids': [[100, 1525, 19, 272, 1]], 'attention_mask': [[1, 1, 1, 1, 1]]}

In [119]:
tokenizer(['This answer is C'])

{'input_ids': [[100, 1525, 19, 205, 1]], 'attention_mask': [[1, 1, 1, 1, 1]]}

In [120]:
tokenizer(['This answer is D'])

{'input_ids': [[100, 1525, 19, 309, 1]], 'attention_mask': [[1, 1, 1, 1, 1]]}

In [121]:
ans_id_dict = {71: "A", 272: "B", 205: "C", 309: "D"}

In [122]:
preds_softmax = torch.nn.functional.softmax(preds_tokenized.logits, dim=-1)
preds_softmax.shape

torch.Size([1, 1, 32128])

In [123]:
answers_prob = []
for t in ans_id_dict.keys():
    answers_prob.append(preds_softmax[...,t])
answers_prob

[tensor([[0.0378]], grad_fn=<SelectBackward0>),
 tensor([[0.0730]], grad_fn=<SelectBackward0>),
 tensor([[0.0606]], grad_fn=<SelectBackward0>),
 tensor([[0.0324]], grad_fn=<SelectBackward0>)]

In [125]:
ref = data_points['train'][0]["answerKey"]
ref

'B'

In [126]:
model_large = "google/flan-t5-large"
tokenizer_large = AutoTokenizer.from_pretrained(model_large)
model_large = AutoModelForSeq2SeqLM.from_pretrained(model_large)

In [ ]:
tokenizer_large(['This answer is A'])

In [ ]:
tokenizer_large(['This answer is B'])


In [ ]:
tokenizer_large(['This answer is C'])


In [ ]:
tokenizer_large(['This answer is D'])


In [318]:
tokenized_datasets = data_points.map(preprocess_function, batched=True)

In [303]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answerKey', 'input_ids', 'attention_mask'],
        num_rows: 2251
    })
    test: Dataset({
        features: ['id', 'question', 'choices', 'answerKey', 'input_ids', 'attention_mask'],
        num_rows: 2376
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answerKey', 'input_ids', 'attention_mask'],
        num_rows: 570
    })
})

In [322]:
tokenized_datasets["train"][2]

{'id': 'Mercury_SC_401169',
 'question': 'When a switch is used in an electrical circuit, the switch can',
 'choices': {'text': ['cause the charge to build.',
   'increase and decrease the voltage.',
   'cause the current to change direction.',
   'stop and start the flow of current.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'D',
 'input_ids': [2625,
  10,
  21,
  284,
  746,
  132,
  33,
  3,
  9,
  360,
  3703,
  13,
  1525,
  5,
  1525,
  48,
  822,
  57,
  4622,
  8,
  200,
  1160,
  10,
  366,
  3,
  9,
  3615,
  19,
  261,
  16,
  46,
  4850,
  4558,
  6,
  8,
  3615,
  54,
  3541,
  32,
  867,
  7,
  10,
  71,
  1137,
  8,
  1567,
  12,
  918,
  5,
  5,
  272,
  993,
  11,
  6313,
  8,
  10594,
  5,
  5,
  205,
  1137,
  8,
  750,
  12,
  483,
  2212,
  5,
  5,
  309,
  1190,
  11,
  456,
  8,
  2537,
  13,
  750,
  5,
  5,
  3,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [131]:
tokenizer_large.decode(tokenized_datasets['train'][0]['input_ids'])

'context: for each questions there are a few choices of answer. answer this question by choosing the best choice: Which factor will most likely cause a person to develop a fever?Choices: A a leg muscle relaxing after exercise. B a bacterial population in the bloodstream. C several viral particles on the skin. D carbohydrates being digested in the stomach. </s>'

In [174]:
train_input_ids = tokenized_datasets["train"]["input_ids"]
large_model_outputs = []
results = {}
group_len = 20
for i in range(0, len(train_input_ids)):
        test_tensor = torch.tensor([train_input_ids[i]])
        preds = model_large(input_ids=test_tensor, decoder_input_ids=torch.tensor([[model_large.config.decoder_start_token_id,]]))      
        preds_prob = []
        for t in ans_id_dict.keys():
            preds_prob.append(torch.nn.functional.softmax(preds.logits, dim=-1)[...,t][0][0].item())
            
        large_model_outputs.append(preds_prob)            

with open("QA_large_model_probability_output.txt", "w") as fp:
    json.dump(large_model_outputs, fp)

In [175]:
large_model_outputs[:5]

[[0.009190092794597149,
  0.7991004586219788,
  0.185978502035141,
  0.004984385333955288],
 [0.017971789464354515,
  0.6050876975059509,
  0.07156172394752502,
  0.3038560748100281],
 [0.008832423016428947,
  0.04372744262218475,
  0.13115499913692474,
  0.8153246641159058],
 [0.8612575531005859,
  0.07377391308546066,
  0.012691803276538849,
  0.05096852034330368],
 [3.0785071203354164e-07,
  1.9637053583210218e-07,
  4.781199436365569e-07,
  4.837812070945802e-07]]

In [320]:
tokenized_datasets["train"] = tokenized_datasets["train"].add_column("labels", large_model_outputs)

In [305]:
model.config.decoder_start_token_id

0

In [309]:
from transformers import DataCollatorForSeq2Seq, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [310]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    result = metric.compute(predictions=predictions, references=labels)
    return result

In [350]:
from  transformers import Trainer, TrainingArguments
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # print(inputs)
        labels = inputs.pop("labels")
        labels = labels[0,...]
        # print(labels)
        # forward pass
        # print(inputs['input_ids'].shape)
        
        # print(model)
        outputs = model(**inputs, decoder_input_ids=torch.tensor([[0]]))
        # print(outputs.logits)
        logits = outputs.get("logits")
        # print(logits.shape)
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = torch.nn.CrossEntropyLoss()
        preds_prod = []
        for i in range(1):
            pred_prob = []
            for t in ans_id_dict.keys():
                pred_prob.append(torch.nn.functional.softmax(logits, dim=-1)[..., t][0][0].item())
                # print(pred_prob)
            preds_prod.append(pred_prob)
            # print(preds_prod)
        # print(labels)
        # print(preds_prod)
        preds = torch.tensor(preds_prod[0]).to("cuda")
        preds.requires_grad_()
        # print(preds)
        loss = loss_fct(preds, labels)
        return (loss, outputs) if return_outputs else loss

In [355]:
training_args = TrainingArguments(
    output_dir="model/flant5_small_lr_10-5_wd_10-2_qa",
    evaluation_strategy="no",
    learning_rate=1e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    save_strategy="epoch",
    num_train_epochs=3,
    # load_best_model_at_end=True,
    
)
# training_args = training_args.set_dataloader(train_batch_size=1, eval_batch_size=1)
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    # eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

trainer.train()

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
500,1.297600
1000,1.303400
1500,1.309500
2000,1.326400
2500,1.305100
3000,1.305700


TrainOutput(global_step=3378, training_loss=1.307694195148294, metrics={'train_runtime': 130.6468, 'train_samples_per_second': 51.689, 'train_steps_per_second': 25.856, 'total_flos': 233404045092864.0, 'train_loss': 1.307694195148294, 'epoch': 3.0})

Total points are 2376. Number of correct answers is 1675. Number of empty answers is 105
